## Влияние ошибок округления на решение СЛАУ. Числа обусловленности.

In [9]:
from numpy import sqrt, abs, array, dot, zeros, transpose
from numpy.linalg import inv, solve, norm, cond, det
from prettytable import PrettyTable

In [10]:
class ConditionNumbers:
    def __init__(self, A):
        self.A = A

    def spectral_criterion(self, variation = 0):
        var_A = self.A - variation
        return norm(var_A)*norm(inv(var_A))

    def volumetric_criterion(self, variation = 0):
        var_A = self.A - variation
        det_A = det(var_A)
        multiplication = 1
        for i in range(len(var_A)):
            sum = 0
            for j in range(len(var_A)):
                sum += var_A[i][j]**2
            multiplication *= sqrt(sum)
        return multiplication/abs(det_A)

    def angle_criterion(self, variation = 0):
        var_A = self.A - variation
        C = inv(var_A)
        return max([norm(a_n)*norm(c_n) for a_n, c_n in zip(var_A, transpose(C))])

In [11]:
def hilbert_matrix(n: int):
    matrix = zeros((n, n))
    for i in range(1, n + 1):
        for j in range(1, n + 1):
            matrix[i - 1][j - 1] = 1 / (i + j - 1)
    return matrix

### Тесты

In [12]:
systems_of_equations = []

#7 вариант из методички Пакулиной
A1 = array([[-402.9, 200.7], [1204.2, -603.6]])
b1 = array([200, -600])
x1 = solve(A1, b1)
systems_of_equations.append({'A': A1, 'b': b1, 'x': x1})

#трехдиагональная матрица с диагональным преобладанием
A2 = array([[5, 4, 0, 0], [2.75, 3.75, 1, 0], [0, 0.5, 7, 6], [0, 0, 2, 10]])
x2 = array([1, 1, 1, 1])
b2 = dot(A2, x2)
systems_of_equations.append({'A': A2, 'b': b2, 'x': x2})

#матрица Гильберта 3х3
A3 = hilbert_matrix(3)
x3 = array([1, 1, 1])
b3 = dot(A3, x3)
systems_of_equations.append({'A': A3, 'b': b3, 'x': x3})

#матрица Гильберта 6х6
A4 = hilbert_matrix(6)
x4 = array([1, 1, 1, 1, 1, 1])
b4 = dot(A4, x4)
systems_of_equations.append({'A': A4, 'b': b4, 'x': x4})

In [13]:
for system in systems_of_equations:
    print('Ax = b, A:')
    print(system['A'])
    print('b:')
    print(system['b'])
    print('Точное решение:')
    print(system['x'])
    cond_num = ConditionNumbers(system['A'])
    print('Спектральный критерий = ', cond_num.spectral_criterion())
    print('Объёмный критерий = ', cond_num.volumetric_criterion())
    print('Угловой критерий = ', cond_num.angle_criterion())
    results = PrettyTable()
    results.field_names = ["вариация", "norm(x-x*)"]
    results.add_row([0.01, norm(system['x'] - solve(system['A'] - 0.01, system['b'] - 0.01))])
    results.add_row([0.0001, norm(system['x'] - solve(system['A'] - 0.0001, system['b'] - 0.0001))])
    results.add_row([0.000001, norm(system['x'] - solve(system['A'] - 0.000001, system['b'] - 0.000001))])
    results.add_row([0.00000001, norm(system['x'] - solve(system['A'] - 0.00000001, system['b'] - 0.00000001))])
    print(results)
    print('')

Ax = b, A:
[[-402.9  200.7]
 [1204.2 -603.6]]
b:
[ 200 -600]
Точное решение:
[-0.19900498  0.59701493]
Спектральный критерий =  1338.0029850746614
Объёмный критерий =  402.2000491345348
Угловой критерий =  402.2000491345347
+----------+-----------------------+
| вариация |       norm(x-x*)      |
+----------+-----------------------+
|   0.01   |  0.00706348433270853  |
|  0.0001  | 7.175324200350381e-05 |
|  1e-06   | 7.176460550686191e-07 |
|  1e-08   | 7.176463868839441e-09 |
+----------+-----------------------+

Ax = b, A:
[[ 5.    4.    0.    0.  ]
 [ 2.75  3.75  1.    0.  ]
 [ 0.    0.5   7.    6.  ]
 [ 0.    0.    2.   10.  ]]
b:
[ 9.   7.5 13.5 12. ]
Точное решение:
[1 1 1 1]
Спектральный критерий =  17.809766919184156
Объёмный критерий =  6.755731664445314
Угловой критерий =  4.171166020796755
+----------+-----------------------+
| вариация |       norm(x-x*)      |
+----------+-----------------------+
|   0.01   |  0.008423006559963546 |
|  0.0001  | 8.390543981454405e-05 |
| 